# Introduction
---

This Jupyter notebook was made with the intent of streamlining generation of PCR Master Mix (MMX) spreadsheets. The specific application this notebook was made in regards to is running DS PCR reactions on library samples before submission to Hartwell for sequencing.

# Instructions for This Notebook
---

1. Open this file in a Jupyter notebook editor of your choice (Python 3.7.7+).

2. Change user variables below:

    * LIB_AND_PRIM_DIR: Should stay the same each time. Meant to point at the directory housing the spreadsheet referenced by LIB_AND_PRIM_XLSX.

    * LIB_AND_PRIM_XLSX: Should stay same each time. Title of spreadsheet file that contains information on all of the vectors (and their associated primer sets) currently in use in our library workflows.

    * INPUT_TSV: Can be changed if so desired, but will need to rename file as well. File in current directory that has information on the samples you would like to include. Tab-separated values file ('\t') with four columns: sample_name, vector, ng/uL, and uL.

        * sample_name: sample name you would like used for the sample on resultant files. Do not include primer name on this, as it will be added later.

        * vector: Vector used/containing your library, to be used for matching to associated primer sets.

        * ng/uL: Current or reported concentration of DNA in sample, as reported in ng per microliter

        * uL: Current volume of sample, as reported in uL.

    * SIZE_OF_LIBRARY: Number of gRNAs included in the library being processed

    * COVERAGE: Number of replicates per gene that has been deemed necessary for representative coverage of the library.

    * EXTRA_RXNS: Extra reactions to account for in volume calculations to account for negative control as well as pipetting errors.

3. Edit the input file (referenced and described as INPUT_TSV above) to reflect your current/desired samples.

4. Run all cells in this notebook.

**Process complete!**

Explanation of file output may be found at the bottom of this notebook. After running the notebook, see the "Follow Up Steps" section after "Output" section to find next experimental steps to perform after generation of the spreadsheets.

In [14]:
import os, sys
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import openpyxl
import datetime
from shutil import copy
from localModules import combine_mmx_output

FILE_SUFFIX = 'tecan_test_pmh'# Goes at end of all filenames
EXTRA_RXNS = 1 # 3 allows one repeat -- 1 allows skipping mmx transfer

CUR_DIR = os.path.abspath('')
LIB_AND_PRIM_DIR = os.path.abspath(r"Z:\ResearchHome\Groups\millergrp\home\common\Screens")
LIB_AND_PRIM_XLSX = os.path.join(LIB_AND_PRIM_DIR, 'lib_vectors_and_primers.xlsx')

INPUT_TSV = os.path.join(CUR_DIR, 'input_pcrmmx_v2.tsv')

UPDATING_PRIMERS = True
FIRST_PRIMER_TO_USE = {
    'pLentiCRISPRv2': 'PC448',
    'pLentiGuide-Puro': 'PC92',
    'PC529_LVA': 'PC529-B06',
    'CROPseq': 'JS26.A01.DS.R8'
}

In [15]:
vectors_df = pd.read_excel(LIB_AND_PRIM_XLSX, sheet_name='vectors',engine='openpyxl')
primers_df = pd.read_excel(LIB_AND_PRIM_XLSX, sheet_name='primers_for_screening',engine='openpyxl')
mrp_df = pd.read_excel(LIB_AND_PRIM_XLSX, sheet_name='most_recent_primer', index_col=0,engine='openpyxl') # "Most recent Primers" df

input_df = pd.read_csv(INPUT_TSV, sep='\t')

print('Most recent primers for each vector set:')
display(mrp_df)

Most recent primers for each vector set:


,most_recent_used
primer_set,
pLentiCRISPRv2,PC474
pLentiGuide-Puro,PC396
PC529_LVA,PC529-B12
CROPseq,JS26.A01.DS.R8


In [16]:
vec_to_set = {} # Use this at some point to convert to handling multiple vectors/sets at once
expmt_primers = {}
input_df['primer_set'] = ''
# If actually using samples from more than one vector at a time, will need to write primers to input_df accordingly
for vector in input_df['vector'].unique():
    count = len(input_df[input_df['vector'] == vector].index)
    print(vectors_df)
    vector_row = vectors_df.index[vectors_df['CAGE#'] == vector][0]
    # print(vector_row)
    primers_col = vectors_df.columns.get_loc('Primers')
    primer_set = vectors_df.iat[vector_row, primers_col]
    vec_to_set[vector] = primer_set
    print(f'{vector} requires {count} primers. (Primer set: {primer_set})\n')
    try:
        expmt_primers[primer_set] += count
    except KeyError:
        expmt_primers[primer_set] = count
    except Exception as e:
        print(e)
    
    input_df.loc[input_df['vector'] == vector, 'primer_set'] = primer_set

# print(vec_to_set)

     CAGE#                                      Name               Digest  \
0   SNP112                        pLentiCRISPRv3-GFP         Esp3I/ BsmBI   
1    SNP36                       pLentiCRISPRv2-Puro         Esp3I/ BsmBI   
2    SNP28                           pLentiGuidePuro         Esp3I/ BsmBI   
3     SP16                          pLentiGuide-eGFP         Esp3I/ BsmBI   
4    PC291  pXPR_502_lentiguide-P65-HSF (Activation)         Esp3I/ BsmBI   
5    PC292          pXPR_modified_TRACR (Inhibition)         Esp3I/ BsmBI   
6    PC529                              LVA-Ametrine         Esp3I/ BsmBI   
7    PC290                        pLentiCRISPRv3-GFP         Esp3I/ BsmBI   
8    PC592                              pCRISPRia-v2           Bstx1/Blp1   
9    PC226                   LMA-Ametrine_retrovirus                 BbsI   
10     NaN                       pLentiCRISPR_v1 (!)                  NaN   
11   PC329                                       NaN         Esp3I/ BsmBI   

In [17]:
# Iterate through vectors found in this experiment's input df and find primers for each
# To actually implement for multiple vectors, will need to assign primers to rows properly
#   in the non-sliced input_df

final_list = []
final_primer_df = pd.DataFrame()

for set, count in expmt_primers.items():
    vset, vcount = (set, count)
    vector_primers = primers_df[primers_df['primer_group'] == vset]
    #print(vector_primers)
    recent_primer = mrp_df.at[vset, 'most_recent_used']
    if not UPDATING_PRIMERS:
        recent_primer = FIRST_PRIMER_TO_USE[vset]
    recent_index = vector_primers.reset_index().set_index('primer_name').loc[recent_primer, 'level_0']
    #print(recent_index)
    primer_df_indices = vector_primers.index.to_list()
    loc_in_primer_df = primer_df_indices.index(recent_index)
    #print(loc_in_primer_df)
    #print(primer_df_indices[loc_in_primer_df])
    shifted_indices = primer_df_indices[loc_in_primer_df + 1:] + primer_df_indices[:loc_in_primer_df]
    shifted_df = vector_primers.loc[shifted_indices]
    # print(shifted_df)
    vprimers_series = shifted_df['primer_name'].head(vcount)
    final_list.append({'set': vset, 'series': vprimers_series})
    
    print(final_list)


[{'set': 'pLentiGuide-Puro', 'series': 46    PC397
47    PC398
Name: primer_name, dtype: object}]


In [18]:
# Begin generating output DataFrame

output_df = input_df.copy(deep=True)

# output_df['primer'] = final_list[0]['list'] # NOTE: This relies on only one vector/primer set being used.
output_df['primer'] = ''
for v in final_list:
    output_df.loc[output_df['primer_set'] == v['set'], ['primer']] = v['series'].values


In [19]:
def dna_to_screen(row):
    return ((row['num_grnas'] * row['coverage']) * (6/1000) / 1000)

output_df['ug_to_screen'] = output_df.apply(dna_to_screen, axis=1)


In [20]:
# TODO: Maybe fix the hartwell sample name to be a formula instead of hard-coded
# As-is, if you make any changes to the sample names or primers, it will not be reflected in the hart_sample_name
# output_df['hart_sample_name'] = output_df['sample_name'].map(str) + '_' + output_df['primer'].map(str)

output_df['total_ug_DNA'] = round(((output_df['ng/uL'] * output_df['uL']) / 1000), 1) # Total ug of DNA for each sample

def uL_for_10ug(ng_per_uL):
    results = round(min(10000/ng_per_uL, 25.0), 1)
    return results

# def enough_dna(total_ug_dna):
#     print(total_ug_dna)
#     return total_ug_dna > DNA_TO_SCREEN
# output_df['total_ug_DNA'].apply(enough_dna)

In [21]:

output_df['uL DNA/rxn'] = output_df['ng/uL'].map(uL_for_10ug) # uL of sample/DNA per reaction
output_df['uL MMX/rxn'] = 75 # 100 - output_df['uL DNA/rxn']
output_df['ug/rxn'] = round((output_df['ng/uL'] * output_df['uL DNA/rxn']) / 1000, 2) # ug of DNA loaded per rxn for each sample
output_df['total_ug_DNA'] = round(((output_df['ng/uL'] * output_df['uL']) / 1000), 1) # Total ug of DNA for each sample

output_df['limit_volume'] = pd.Series(output_df['uL'] / output_df['uL DNA/rxn']).map(np.floor) # Max rxns as limited by volume
output_df['limit_DNA'] = pd.Series(output_df['total_ug_DNA'] / output_df['ug/rxn']).map(np.floor) # Max rxns as limited by DNA
output_df['required_DNA'] = pd.Series(output_df['ug_to_screen'] / output_df['ug/rxn']).map(np.ceil) # Minimum rxns required to reach desired coverage

output_df['n'] = pd.Series(output_df[['limit_volume', 'limit_DNA', 'required_DNA']].min(axis=1)).map(np.ceil) # Finds minimum between max by vol, max by ug DNA, required DNA
output_df['ug_screened'] = pd.Series(output_df['ug/rxn'] * output_df['n']) # Total ug of DNA screened
output_df['n_total'] = output_df['n'].map(int) + EXTRA_RXNS

In [22]:
# Create Master Mix (MMX) series for 1x Rxn (mmx_per_rxn) and for all rxns (gen_mmx)

mmx_per_rxn = pd.Series()
mmx_per_rxn['Rxn Volume'] = 100
mmx_per_rxn['dNTPs (2.5mM)'] = 8
mmx_per_rxn['DMSO'] = 5
mmx_per_rxn['10X Titanium Taq Buffer'] = mmx_per_rxn['Rxn Volume'] / 10
mmx_per_rxn['PC84-PC91 (10uM)'] = 5
mmx_per_rxn['Unique Reverse Primer (10uM)'] = 5
mmx_per_rxn['Titanium Taq'] = 1.5

# max_sample_volume = output_df['uL DNA/rxn'].max()
# mmx_per_rxn['h2o'] = mmx_per_rxn['Rxn Volume'] - (mmx_per_rxn.iloc[1:].sum() + max_sample_volume)
mmx_per_rxn['h2o'] = 40.5 # Assuming all rxns will need 25 uL of DNA+H2O
# display(big_master_mix)

total_rxns = output_df['n_total'].sum()
safe_total = total_rxns * 1.05
# print(total_rxns)
# print(safe_total)

gen_mmx = mmx_per_rxn[1:] * safe_total
gen_mmx = gen_mmx.drop('Unique Reverse Primer (10uM)')
gen_mmx['uL of MMX/rxn'] = gen_mmx.sum() / safe_total
gen_mmx['Total Volume'] = gen_mmx[~gen_mmx.index.isin(['uL of MMX/rxn'])].sum()
# display(big_master_mix_final)

output_df['uL MMX for Individual MMX'] = gen_mmx['uL of MMX/rxn'] * output_df['n_total']

def water_over_min(uL_DNA_per_rxn):
    h2o_to_add_per_rxn = 25 - uL_DNA_per_rxn
    return h2o_to_add_per_rxn

output_df['uL H2O/rxn'] = output_df['uL DNA/rxn'].map(water_over_min) # * output_df['n_total'] # Uncomment previous multiplication to get water for mmx
output_df['uL Rev Primer for individual MMX'] = output_df['n_total'] * mmx_per_rxn['Unique Reverse Primer (10uM)']

# TODO: Make two columns below only run for reactions where n > 1 (since these are already covered by original test PCRs)
# output_df['uL to Remove'] = (EXTRA_RXNS - 2) * (100 - output_df['uL DNA/rxn']) # Reasoning for this calculation is described in Appendix A
output_df['desired MMX vol'] = (output_df['n'] - 1) * 75 # * (100 - output_df['uL DNA/rxn']) # Based on Appendix A -- Desired rxns in MMX * uL DNA/rxn in MMX

output_df['uL DNA to Add'] = (output_df['n'] - 1) * output_df['uL DNA/rxn']
output_df['uL H2O to Add'] = (output_df['n'] - 1) * output_df['uL H2O/rxn']


c:\Users\phall3\Anaconda3\envs\cage37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
# Determine wells to load samples in
# according to Hartwell's preference of A01, B01, C01... F12, G12, H12

well_rows = 'ABCDEFGH'
well_cols = [
    f'{c + 1:02d}' for c in range(12)
]
combs = [r + c for c in well_cols for r in well_rows]

output_df['hart_well'] = combs[:len(output_df)]

# print(well_rows)
# print(well_cols)
# print(combs)
# display(output_df)

In [24]:

today = datetime.date.today().strftime('%Y_%m_%d')
print(f'Today is: {today}\n')

# Make the output directory.
output_dir = os.path.join(CUR_DIR, 'output', f'{today}_{FILE_SUFFIX}')
try:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
except Exception as err:
    print(err)

# Back up old lib_vectors_and_primers file, update file with new most-recent primers
try:
    copy(LIB_AND_PRIM_XLSX, os.path.join(LIB_AND_PRIM_DIR, 'archive', 'lib_vectors_and_primers', f'{today}.xlsx'))
    print('Successfully backed up lib_vectors_and_primers and updated MRP sheet.')
    
    integer_ind_mrp_df = mrp_df.reset_index() # Resets the mrp_df index to be integers instead of the primer set names.
    
    wb = openpyxl.load_workbook(LIB_AND_PRIM_XLSX)
    ws = wb['most_recent_primer']
    
    for set_object in final_list:
        new_most_recent = set_object['series'].iat[-1]
        #display(new_most_recent, old_most_recent)
        old_most_recent = mrp_df.at[set_object['set'], 'most_recent_used']
        if not UPDATING_PRIMERS:
            new_most_recent = old_most_recent
        mrp_index_to_add = integer_ind_mrp_df.loc[
            integer_ind_mrp_df['primer_set'] == set_object['set'] # Find row relating to current primer set
            ].index.to_list()[0] 
        #display(f"mrp_index_to_add {mrp_index_to_add}")# Get the index as a list, then return first (only) element of the list.
        cell = 'B' + str(2 + mrp_index_to_add) # First data cell is B2. Add to row the 0-indexed position in mrp_df.
        ws[cell] = new_most_recent
        new_prime = cell
        #display(new_prime)
        
        print(f'Most recent primer for set  [ {set_object["set"]} ]  has changed.')
        print(f'    FROM:  [ {old_most_recent} ]')
        print(f'      TO:  [ {new_most_recent} ]\n')
    wb.save(LIB_AND_PRIM_XLSX)
    wb.close()
    display(final_list)
except Exception as err:
    print('\n\n!!! Failed to back up lib_vectors_and_primers! Not saving over !!!\n')
    print(err)
    print('\n')
    print('For manual entry:')
    print(f'    Most recent primer for set  [ {final_list[0]["set"]} ]  should change...')
    print(f'        FROM:  [ {old_most_recent} ]')
    print(f'          TO:  [ {new_most_recent} ]\n\n')

# Output the input file for logging
try:
    input_df.to_csv(os.path.join(output_dir, f'{today}_lib_analysis_input.tsv'), sep='\t', index=False)
    print('Successfully saved input DataFrame to TSV.')
except Exception as err:
    print('\n\n!!! Failed to save Input DF to TSV !!!\n')
    print(err)
    print('\n')

# Output the generic master mix table
try:
    gen_mmx.to_csv(os.path.join(output_dir, f'{today}_lib_analysis_mmx_gen.tsv'), sep='\t', index=True, header=True)
    print('Successfully saved Gen MMX to TSV.')
except Exception as err:
    print('\n\n!!! Failed to save Gen MMX to TSV !!!\n')
    print(err)
    print('\n')

# Output the full dataframe table
try:
    output_df.to_csv(os.path.join(output_dir, f'{today}_lib_analysis_mmx_full.tsv'), sep='\t', index=False)
    print('Successfully saved full DataFrame to TSV.')
except Exception as err:
    print('\n\n!!! Failed to save main DataFrame to TSV !!!\n')
    print(err)
    print('\n')

# Output the simplified DF table
simplified_output_df = output_df[[
    'sample_name',
    'n_total',
    'uL MMX for Individual MMX',
    'primer',
    'uL Rev Primer for individual MMX',
    # 'uL MMX/rxn',
    'uL DNA/rxn',
    'uL H2O/rxn',
    # 'uL to Remove',
    'desired MMX vol',
    'uL H2O to Add',
    'uL DNA to Add',
    'hart_well',
    ]]
try:
    simplified_output_df.to_csv(os.path.join(output_dir, f'{today}_lib_analysis_mmx_simplified.tsv'), sep='\t', index=False)
    print('Successfully saved simplified DataFrame to TSV.')
except Exception as err:
    print('\n\n!!! Failed to save main DataFrame to TSV !!!\n')
    print(err)
    print('\n')

print(f'\nOutput directory is:\n    {output_dir}')

# TODO: Ask if they want you to copy somewhere, and take a Windows path as input
#   Try/Except copying output_dir/today to desired destination

Today is: 2025_01_17

Successfully backed up lib_vectors_and_primers and updated MRP sheet.
Most recent primer for set  [ pLentiGuide-Puro ]  has changed.
    FROM:  [ PC396 ]
      TO:  [ PC398 ]



[{'set': 'pLentiGuide-Puro',
  'series': 46    PC397
  47    PC398
  Name: primer_name, dtype: object}]

Successfully saved input DataFrame to TSV.
Successfully saved Gen MMX to TSV.
Successfully saved full DataFrame to TSV.
Successfully saved simplified DataFrame to TSV.

Output directory is:
    z:\ResearchHome\Groups\millergrp\projects\CrisprDesigns\common\screens\programs\pcr_mmx\output\2025_01_17_Lib122_Screen_Mack_841671


In [25]:
list_of_names = ["Input", "Full Table", "Generic MMX", "Simplified Table"]
list_of_dfs = [input_df, output_df, gen_mmx, simplified_output_df]
list_of_sheets = ["input", "pcr_mmx", "gen_mmx", "pcr_mmx"]
list_of_calc_cols = [[], [2, 3], [1], [2]]

mmx_obj_list = combine_mmx_output.make_list_of_MMXOutputFiles(
    list_of_names, list_of_dfs, list_of_sheets, list_of_calc_cols)

combine_mmx_output.make_excel(mmx_obj_list, os.path.join(output_dir, f'{today}_mmx-{FILE_SUFFIX}.xlsx'))

In [26]:
# Reporting

# TODO: Generate a short summary file containing relevant information

# print('\nGen MMX:')
# display(gen_mmx)

# print('\nFinal volumes:')
# display(output_df[['sample_name', 'hart_sample_name', 'n_total', 'uL MMX for Individual MMX', 'uL H2O for individual Master Mix', 'uL Rev Primer for individual MMX']])

# Output
---

Expected output of this file will be two TSV files containing the original columns from the input TSV, as well as the following additional columns:

* primer: Generated primer choice for sample

* hart_sample_name: Sample name to be used in submission to Hartwell

* uL DNA/rxn: uL of DNA that will be used per PCR reaction

* ug/rxn: ug of DNA that will be used per PCR reaction

* total_ug_DNA: Sum total of ug DNA that will be processed by all PCR reactions associated with the sample (n)

* limit_volume: Maximum reactions that may be generated, as limited by the **volume** of sample available

* limit_DNA: Maximum reactions that may be generated, as limited by the **DNA** of sample available

* n: Number of reactions to be made from the sample's individual master mix

    * Determined by finding the lowest number out of: maximum by volume, maximum by DNA, minimum to process appropriate ug of DNA for representative analysis

* n_total: Total number of reactions that may be fulfilled by the individual master mix (n + EXTRA_RXNS)

* uL MMX for Individual MMX: uL to transfer from the large/generic master mix into a new tube for the sample's individual master mix

* uL H2O for individual Master Mix: uL of H2O to add to the individual master mix before aliquoting into reactions

* uL Rev Primer for individual MMX: uL of individual's designated reverse primer to add to individual MMX before aliquoting into reactions

* uL to Remove: uL of individual MMX to remove after rxn confirmation
    
    * **WARNING!** Only remove this volume **AFTER** confirming test reactions and **BEFORE** adding DNA to the MMX.

* uL DNA to Add: After confirming test rxns and removing the uL of MMX from above, add this many uL of DNA to the individual MMX to allow equal mixing and distribution to PCR reactions by a single dispense rather than dispensing DNA to each well, then dispensing MMX to each well.
    
    * **WARNING!** Only add uL of DNA **AFTER** reaction confirmation by gel and **AFTER** removing the "uL to Remove" from individual MMX


# Follow-Up Steps
---

After running this notebook, use the newly-generated spreadsheets to follow the steps below.



## Prep and Label Tubes

Begin by getting all of the tubes you will need ready for individual master mixes. Label as necessary.

> ### ***Pro Tip!***

>> 1.5mL Tubes work for n_total up to 15, 15mL up to 150.

>> ...I hope for your sake you don't have more reactions to perform for a sample than 150.

## Generic Master Mix

1. Use the bigMMX table to determine volumes of reagents to retrieve/begin thawing.

2. Prepare and label a tube for the generic MMX with a max volume greater than "Total Volume" in the bigMMX table.

    * Note: "uL of MMX/rxn" is mostly just for reference; It is the volume of Generic MMX that will be present in each individual PCR reaction.

3. Add H2O to the tube, then proceed through remaining ingredients in order.



## Making individual MMXs

1. Add Generic MMX for each sample to the individual MMX tubes in the volume listed in the "uL MMX for Individual MMX" column in the mmx table.

2. Add H2O according to "uL H2O for Individual MMX" column to the individual MMX.

3. Add the selected reverse primer for the sample in the volume listed as "uL Reverse Primer for Individual MMX"



## Running Test Reactions

1. Aliquot one reaction for the negative control and one for the positive control into PCR tubes.

    1.1. This volume will be $[Reaction Volume] - [uL DNA/rxn]$, so typically 100 uL minus however many uL of DNA you add.

2. Add H2O to the negative control and DNA to the positive control to bring the reactions to 100 uL.

3. Cap tubes, run PCR reaction, and run small amount (2 uL) on a gel to check for bands.

    3.1. If bands are present, continue to final steps.
    
    3.2. If not, either determine which step to start over from, or start over from before you made the generic MMX.



## Final Steps

1. Remove "uL to Remove" from the individual MMXs as necessary according to the mmx table.

2. Add "uL DNA to Add" DNA from the sample to the individual MMX according to the mmx table.

3. Aliquot 100 uL per well to $n - 1$ wells to perform the remaining sample PCRs. (No additional neg. control necessary.)

4. Run PCRs.

5. Label plate on side with notched corner in format: " $[DATE]\ \ \ \ SRM: [SRM Order Number]\ \ \ \ CRISPR$ "

    5.1. **Use Sharpie!** Hartwell specifically requested Sharpie rather than adhesive label to avoid lost labels.

# Appendix
---

## Appendix A -- "uL to Remove"
---

### Description

This value represents the value of uL that should be removed from the individual master mix before final aliquots. That is to say, before adding DNA to the master mix and subsequent aliquoting of the remaining PCR reactions required to have appropriate representation of the sample.

Put simply, this value should be as follows:

> $[uL \ \ to \ \ Remove] = [Rxns \ \ to \ \ remove] * [uL \ \ of \ \ MMX/rxn]$

### Variable A: Reactions to Remove

#### Equation

$[Rxns \ \ to \ \ remove] = [current \ \ rxns \ \ in \ \ MMX] - [desired \ \ rxns]$

#### Current Reactions in MMX:

> $[current \ \ rxns] = [n\_total] - [number \ \ of \ \ test \ \ rxns]$

> $\ => [current \ \ rxns] = n\_total - 2 (neg. \ \ control \ \ and \ \ sample)$

#### Desired Reactions:

> $[desired \ \ rxns] = [remaining \ \ necessary \ \ PCRs] + [desired \ \ excess]$

> $\ => [desired \ \ rxns] = (n - 1) + 1 = n$

> $\ => [desired \ \ rxns] = n$

#### Simplifying Rxns to Remove:

> $[rxns \ \ to \ \ remove] = [current \ \ rxns] - [desired \ \ rxns]$

> $\ => (n\_total - 2) - (n) = n\_total - 2 - n$

Because diff b/t n_total and n = EXTRA_RXNS, $(n\_total - 2 - n)$ can be simplified to...

> $[rxns \ \ to \ \ remove] = EXTRA\_RXNS - 2$

### Variable B: uL of MMX Per Reaction

Because individual MMX now contains all components except for DNA...

> $[uL DNA/rxn \ \ in \ \ MMX] = [rxn \ \ volume] - [missing \ \ volume \ \ of \ \ DNA]$

> $\ => [uL DNA/rxn \ \ in \ \ MMX] = 100 - [uL \ \ DNA \ \ to \ \ Add]$

### SUMMARY

> $[uL \ \ to \ \ Remove] = [rxns \ \ to \ \ remove] * [uL DNA/rxn \ \ in \ \ MMX]$

> $[rxns \ \ to \ \ remove] = EXTRA\_RXNS - 2$

> $[uL DNA/rxn \ \ in \ \ MMX] = 100 - [uL \ \ DNA/rxn]$

Therefore:

> $[uL \ \ to \ \ Remove] = (EXTRA\_RXNS - 2) * (100 - [uL \ \ DNA/rxn])$